In [4]:
# importing prerequisites
import sys
import requests
import tarfile
import json
import numpy as np
from os import path
from PIL import Image
from PIL import ImageFont, ImageDraw
from glob import glob
from matplotlib import pyplot as plt
%matplotlib inline

In [5]:

fname = 'examples.tar.gz'
url = 'https://dax-cdn.cdn.appdomain.cloud/dax-pubtabnet/1.0.0/' + fname
r = requests.get(url)
open(fname , 'wb').write(r.content)

380823

In [8]:
tar = tarfile.open(r'C:\Users\sande\OneDrive\Desktop\res+gpt\table-recognition\code\image_caption\dataset\examples.tar.gz')
tar.extractall()
tar.close()


In [9]:
# Helper function to read in tables from the annotations
import re
from bs4 import BeautifulSoup as bs

def format_html(img):
    ''' Formats HTML code from tokenized annotation of img
    '''
    html_string = '''<html>
                     <head>
                     <meta charset="UTF-8">
                     <style>
                     table, th, td {
                       border: 1px solid black;
                       font-size: 10px;
                     }
                     </style>
                     </head>
                     <body>
                     <table frame="hsides" rules="groups" width="100%%">
                         %s
                     </table>
                     </body>
                     </html>''' % ''.join(img['html']['structure']['tokens'])
    cell_nodes = list(re.finditer(r'(<td[^<>]*>)(</td>)', html_string))
    assert len(cell_nodes) == len(img['html']['cells']), 'Number of cells defined in tags does not match the length of cells'
    cells = [''.join(c['tokens']) for c in img['html']['cells']]
    offset = 0
    for n, cell in zip(cell_nodes, cells):
        html_string = html_string[:n.end(1) + offset] + cell + html_string[n.start(2) + offset:]
        offset += len(cell)
    # prettify the html
    soup = bs(html_string)
    html_string = soup.prettify()
    return html_string

In [ ]:
# Loading the json annotations
with open('/dataset/examples/PubTabNet_Examples.json', 'r') as fp:
    annotations = json.load(fp)

In [ ]:
# Inspecting the annotations
annotations.keys()

In [ ]:
annotations['images'][0].keys()


In [ ]:
annotations['images'][0]

In [ ]:
# Extracting the HTML for the table from the annotation
html_string = format_html(annotations['images'][0])
print(html_string)

In [ ]:
# Rendering the above HTML in Jupyter Notebook for a more readable format
from IPython.core.display import display, HTML
display(HTML(html_string))